In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from datasets import load_dataset
eli5 = load_dataset("eli5", split="train_asks[:5000]")
eli5 = eli5.train_test_split(test_size=0.2)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
eli5 = eli5.flatten()

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [3]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=eli5["train"].column_names,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1032 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=1)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
class Dataset(Dataset):
    def __init__(self,lm_dataset):
        self.lm_dataset=lm_dataset
        self.input_ids=[torch.tensor(x) for x in self.lm_dataset['train']['input_ids']]
        self.labels=[torch.tensor(x) for x in self.lm_dataset['train']['labels']]
        
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Tokenize and encode the sequence
        return self.input_ids[idx], self.labels[idx]

In [6]:
dataset=Dataset(lm_dataset)
dataset[0]

(tensor([18712,   263,    11,   475,   351,   517,  1176,    13,   220,   198,
          3886,  4395,   606,    13,   220,   198, 23379,   477, 45656,   389,
           783,  9857,   832, 39290, 40990, 34628,    11, 14821, 44847, 34628,
           290, 14821, 22969,   357,  1462,  1438,   257,  1178,   737,   220,
           198, 48401,  5479,   481,  3729,  1663,    11,   475,  2192,   407,
           287, 37312,   290, 22303,   262,   835,   530,   561,  1607,    13,
           383,  1103, 27580,   318,   379,   262, 41858,   290, 15709, 48228,
          1241,    13,   220,   198, 30374,   779,   534, 12855,   319,   534,
          3797,    13,   220,   198,  5297,  1682,    11, 13527,  5107,   357,
          1350,  3243,  4569, 12822, 31562,     8,   286,  8136,   284,   262,
          6769, 16022,  7800,  3499, 12779,   326,  1249,   884,  3499, 12779,
           355, 12855, 26741,   326, 19396,   287,  3095,  1633,    13,   220,
           198,    43, 19865,   481,   307,  8347,  

In [7]:
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

dataloader = DataLoader(dataset, batch_size=8,shuffle=True)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

dataloader

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
from tqdm import tqdm
import gc

In [10]:
torch.cuda.empty_cache()
gc.collect()

0

In [11]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inp, lab in tqdm(dataloader):
        inputs = inp.to(device)
        labels = lab.to(device)

        # Forward pass
        outputs = model(inputs, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')


100%|██████████████████████████████████████████████████████████████████████████████| 1106/1106 [02:49<00:00,  6.53it/s]


Epoch 1/100, Loss: 4.029330241960384


100%|██████████████████████████████████████████████████████████████████████████████| 1106/1106 [02:48<00:00,  6.57it/s]


Epoch 2/100, Loss: 3.573347851337619


100%|██████████████████████████████████████████████████████████████████████████████| 1106/1106 [02:48<00:00,  6.55it/s]


...

Epoch 98/100, Loss: 0.12432467264492085


100%|██████████████████████████████████████████████████████████████████████████████| 1106/1106 [02:49<00:00,  6.53it/s]


Epoch 99/100, Loss: 0.12273606695032681


100%|██████████████████████████████████████████████████████████████████████████████| 1106/1106 [02:49<00:00,  6.52it/s]

Epoch 100/100, Loss: 0.1223069200693052


In [12]:
torch.save(model, 'example_distiledGPT.pth')